## 2026 EY AI & Data Challenge - TerraClimate Data Extraction Notebook

This notebooks demonstrates how to access the TerraClimate dataset. TerraClimate is a dataset of monthly climate and climatic water balance for global terrestrial surfaces from 1958 to the present. These data provide important inputs for ecological and hydrological studies at global scales that require high spatial resolution and time-varying data. All data have monthly temporal resolution and a ~4-km (1/24th degree) spatial resolution. This dataset is provided in Zarr format. 

For more information, visit: [terraclimate- overview](https://planetarycomputer.microsoft.com/dataset/terraclimate#overview) 

## Load In Dependencies
The following code installs the required Python libraries (found in the requirements.txt file) in the Snowflake environment to allow successful execution of the remaining notebook code. After running this code for the first time, it is required to “restart” the kernal so the Python libraries are available in the environment. This is done by selecting the “Connected” menu above the notebook (next to “Run all”) and selecting the “restart kernal” link. Subsequent runs of the notebook do not require this “restart” process.

In [ ]:
!pip install uv
!uv pip install  -r requirements.txt 

In [1]:
import snowflake
from snowflake.snowpark.context import get_active_session
session = get_active_session()

import warnings
warnings.filterwarnings("ignore")

# Data manipulation and analysis
import numpy as np
import pandas as pd

# Multi-dimensional arrays and datasets (e.g., NetCDF, Zarr)
import xarray as xr

from scipy.spatial import cKDTree

# Planetary Computer tools for STAC API access and authentication
import pystac_client
import planetary_computer as pc

from datetime import date
from tqdm import tqdm
import os

## Extracting TerraClimate Data Using API Calls

The API-based method allows us to efficiently access **TerraClimate** data for specific regions and time periods through the [Microsoft Planetary Computer](https://planetarycomputer.microsoft.com/), ensuring scalability and reproducibility of the process.

Through the API, we can extract climate variables such as **Potential Evapotranspiration (PET)**, which represents the atmospheric demand for water. This variable provides important insights into surface moisture balance and helps improve the accuracy of water quality modeling.

This approach ensures consistent, automated retrieval of high-resolution climate data that can be easily integrated with satellite-derived features for comprehensive environmental and hydrological analysis.


### Loading and Mapping TerraClimate Data

This section demonstrates how **TerraClimate climate variables**, such as **Potential Evapotranspiration (PET)**, are loaded and mapped to sampling locations.

- The **load_terraclimate_dataset** function opens the TerraClimate Zarr/NetCDF dataset from the Microsoft Planetary Computer, handling storage options automatically.
- The **filterg** function filters the dataset for the desired time range (2011–2015) and the spatial extent corresponding to the study region. The resulting data is converted into a pandas DataFrame with standardized column names.
- The **assign_nearest_climate** function maps each sampling location to its **nearest TerraClimate grid point** using a KD-tree and assigns the climate variable values corresponding to the closest timestamp.

This workflow ensures efficient, reproducible retrieval of climate variables, while allowing participants to work with pre-extracted CSV files for faster benchmarking and analysis.


In [2]:
def load_terraclimate_dataset():
    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=pc.sign_inplace,
    )
    collection = catalog.get_collection("terraclimate")
    asset = collection.assets["zarr-abfs"]

    if "xarray:storage_options" in asset.extra_fields:
        ds = xr.open_zarr(
            asset.href,
            storage_options=asset.extra_fields["xarray:storage_options"],
            consolidated=True,
        )
    else:
        ds = xr.open_dataset(
            asset.href,
            **asset.extra_fields["xarray:open_kwargs"],
        )

    return ds

In [3]:
# --- Filtering function (kept identical) ---
# def filterg(ds, var):
#     ds_2011_2015 = ds[var].sel(time=slice("2011-01-01", "2015-12-31"))

#     df_var_append = []
#     for i in tqdm(range(len(ds_2011_2015.time))):
#         df_var = ds_2011_2015.isel(time=i).to_dataframe().reset_index()
#         df_var_filter = df_var[
#             (df_var['lat'] > -35.18) & (df_var['lat'] < -21.72) &
#             (df_var['lon'] > 14.97) & (df_var['lon'] < 32.79)
#         ]
#         df_var_append.append(df_var_filter)

#     df_var_final = pd.concat(df_var_append, ignore_index=True)
#     print(f"Filtering for {var} completed")

#     df_var_final['time'] = df_var_final['time'].astype(str)

#     # Column mapping
#     col_mapping = {"lat": "Latitude", "lon": "Longitude", "time": "Sample Date"}
#     df_var_final = df_var_final.rename(columns=col_mapping)

#     return df_var_final
def filterg(ds, var):
    da = ds[var].sel(time=slice("2011-01-01", "2015-12-31"))

    # bbox filter（跟你原本一致）
    da = da.where(
        (da["lat"] > -35.18) & (da["lat"] < -21.72) &
        (da["lon"] > 14.97)  & (da["lon"] < 32.79),
        drop=True
    )

    df = da.to_dataframe().reset_index()
    df["time"] = df["time"].astype(str)

    df = df.rename(columns={"lat": "Latitude", "lon": "Longitude", "time": "Sample Date"})
    print(f"Filtering for {var} completed, rows={len(df)}")
    return df


In [4]:
# --- Climate variable assignment function (unchanged logic) ---
def assign_nearest_climate(sa_df, climate_df, var_name):
    """
    Map nearest climate variable values to a new DataFrame 
    containing only the specified variable column.
    """
    sa_coords = np.radians(sa_df[['Latitude', 'Longitude']].values)
    climate_coords = np.radians(climate_df[['Latitude', 'Longitude']].values)

    tree = cKDTree(climate_coords)
    dist, idx = tree.query(sa_coords, k=1)

    nearest_points = climate_df.iloc[idx].reset_index(drop=True)

    sa_df = sa_df.reset_index(drop=True)
    sa_df[['nearest_lat', 'nearest_lon']] = nearest_points[['Latitude', 'Longitude']]

    sa_df['Sample Date'] = pd.to_datetime(sa_df['Sample Date'], dayfirst=True, errors='coerce')
    climate_df['Sample Date'] = pd.to_datetime(climate_df['Sample Date'], dayfirst=True, errors='coerce')

    climate_values = []

    for i in tqdm(range(len(sa_df)), desc=f"Mapping {var_name.upper()} values"):
        sample_date = sa_df.loc[i, 'Sample Date']
        nearest_lat = sa_df.loc[i, 'nearest_lat']
        nearest_lon = sa_df.loc[i, 'nearest_lon']

        subset = climate_df[
            (climate_df['Latitude'] == nearest_lat) &
            (climate_df['Longitude'] == nearest_lon)
        ]

        if subset.empty:
            climate_values.append(np.nan)
            continue

        nearest_idx = (subset['Sample Date'] - sample_date).abs().idxmin()
        climate_values.append(subset.loc[nearest_idx, var_name])

    output_df = pd.DataFrame({var_name: climate_values})

    
    return output_df

### Extracting features for the training dataset

In [5]:
Water_Quality_df = pd.read_csv("water_quality_training_dataset.csv")
display(Water_Quality_df.head(5))

In [6]:
Water_Quality_df.shape

In [7]:
# # Load TerraClimate dataset, filter (time,region,parameter), filter for nearest parameter values
# ds = load_terraclimate_dataset()
# tc_parameter = filterg(ds,'pet')
# Terraclimate_training_df = assign_nearest_climate(Water_Quality_df, tc_parameter, 'pet')
# Load TerraClimate dataset
ds = load_terraclimate_dataset()

# 想抽的變數（照你截圖）
tc_vars = [
    "aet", "def", "pdsi", "pet", "ppt",
    "q", "soil", "srad", "swe",
    "tmax", "tmin",
    "vap", "vpd", "ws",
    "ppt_station_influence", "tmax_station_influence", "tmin_station_influence", "vap_station_influence"
]

# 檢查 dataset 真的有哪些變數
available = set(ds.data_vars)
tc_vars_exist = [v for v in tc_vars if v in available]
tc_vars_missing = [v for v in tc_vars if v not in available]

print("Will extract:", tc_vars_exist)
print("Missing in dataset:", tc_vars_missing)

# 逐一抽取並對應到每筆 Water_Quality_df
Terraclimate_training_df = Water_Quality_df[["Latitude", "Longitude", "Sample Date"]].copy()

for i, v in enumerate(tc_vars_exist):
    if i % 3 == 0:                      # 每 3 個變數 refresh token
        ds = load_terraclimate_dataset()
    tc_parameter = filterg(ds, v)
    Terraclimate_training_df[v] = assign_nearest_climate(Water_Quality_df, tc_parameter, v)[v]

## Feature Engineering: 數值轉換、交互項 & 氣候距平

此區塊針對已抓取的 TerraClimate 變數進行三類特徵工程，分別對應 `add_basic_derived`、`add_interactions`、`add_anomalies` 三個函數：

**一、數值轉換（`add_basic_derived`）**
- Log1p 轉換：對右偏分布的水文變數（ppt, q, pet, aet）取 log(1+x)，降低極端值影響
- 日夜溫差 DTR：tmax − tmin，反映蒸發與水中溶氧動態
- 水分虧缺比例（water_stress_ratio）：def / pet，比 def 絕對值更穩定的乾旱代理指標

**二、交互項（`add_interactions`）**
- `aet_pet_ratio`：實際/潛在蒸散比 → 地表乾濕程度
- `ppt_x_q`：降雨 × 逕流 → 沖刷污染效應（濁度、大腸桿菌等）
- `tmax_x_vpd`：溫度 × 蒸氣壓差 → 藻類生長風險（悶熱條件）
- `soil_x_ppt`：土壤水分 × 降雨 → 農業面源污染（土壤飽和時徑流帶走污染物）
- `runoff_ratio`：逕流/降雨比 → 集水區產流能力
- `srad_x_tmax`：太陽輻射 × 最高溫 → 水體優養化代理指標

**四、氣候距平（`add_anomalies`）**
- 對每個採樣點、每個月份，計算觀測值與歷史同月（全資料集 1958–present）均值的差距
- 距平 = 當月值 − 該格點歷史同月平均，數值代表「偏離正常氣候」的程度
- 適用變數：ppt, pet, aet, def, pdsi, q, soil, tmax, tmin, vpd, srad
- 距平比絕對值更能捕捉異常氣候事件對水質的衝擊（例如超乾旱月、極端暴雨）


In [ ]:
# ===========================================================
# Feature Engineering 函數定義
# ===========================================================

def add_basic_derived(df):
    """
    一、數值轉換（單變數）
      - log1p 轉換：ppt, q, pet, aet
      - 日夜溫差 DTR
      - 水分虧缺比例 water_stress_ratio
    """
    df = df.copy()

    for col in ['ppt', 'q', 'pet', 'aet']:
        if col in df.columns:
            df[f'log1p_{col}'] = np.log1p(df[col].clip(lower=0))

    if 'tmax' in df.columns and 'tmin' in df.columns:
        df['dtr'] = df['tmax'] - df['tmin']

    if 'def' in df.columns and 'pet' in df.columns:
        df['water_stress_ratio'] = df['def'] / (df['pet'] + 1e-6)

    print(f"[add_basic_derived] 完成，目前欄數：{df.shape[1]}")
    return df


def add_interactions(df):
    """
    二、交互項（水文生態意義明確的組合特徵）
      - aet_pet_ratio, ppt_x_q, tmax_x_vpd
      - soil_x_ppt, runoff_ratio, srad_x_tmax
    """
    df = df.copy()

    if 'aet' in df.columns and 'pet' in df.columns:
        df['aet_pet_ratio'] = df['aet'] / (df['pet'] + 1e-6)

    if 'ppt' in df.columns and 'q' in df.columns:
        df['ppt_x_q'] = df['ppt'] * df['q']

    if 'tmax' in df.columns and 'vpd' in df.columns:
        df['tmax_x_vpd'] = df['tmax'] * df['vpd']

    if 'soil' in df.columns and 'ppt' in df.columns:
        df['soil_x_ppt'] = df['soil'] * df['ppt']

    if 'q' in df.columns and 'ppt' in df.columns:
        df['runoff_ratio'] = df['q'] / (df['ppt'] + 1e-6)

    if 'srad' in df.columns and 'tmax' in df.columns:
        df['srad_x_tmax'] = df['srad'] * df['tmax']

    print(f"[add_interactions] 完成，目前欄數：{df.shape[1]}")
    return df


def add_anomalies(df, ds, vars_list):
    """
    四、氣候距平特徵
    對每個採樣點的每個變數，計算：
        距平 = 當月值 − 該格點歷史同月（全年份）均值
    
    Parameters
    ----------
    df       : pd.DataFrame，含 Latitude, Longitude, Sample Date 及已抓取的氣候欄位
    ds       : xarray.Dataset，TerraClimate 完整資料集（由 load_terraclimate_dataset() 取得）
    vars_list: list[str]，要計算距平的變數名稱清單
    
    Returns
    -------
    pd.DataFrame，原欄位 + 各變數的 _anom 欄位
    """
    df = df.copy()
    df['_month'] = pd.to_datetime(df['Sample Date'], dayfirst=True, errors='coerce').dt.month

    # 計算每個格點 × 每月的歷史均值（climatology）
    # 使用 ds 全年份資料，groupby month 取均值
    for var in vars_list:
        if var not in df.columns:
            print(f"  [skip] {var} 不在 df 中，略過")
            continue
        if var not in ds.data_vars:
            print(f"  [skip] {var} 不在 ds 中，略過")
            continue

        print(f"  計算 {var} 距平...")

        # 從 ds 計算歷史月均值（所有年份，按月份 groupby）
        da = ds[var]
        # 依月份分組後平均 → shape: (12, lat, lon)
        clim = da.groupby('time.month').mean('time')

        # 轉成 DataFrame 便於查表
        clim_df = clim.to_dataframe().reset_index()
        clim_df = clim_df.rename(columns={'lat': 'Latitude', 'lon': 'Longitude', var: f'{var}_clim'})

        # 對每筆樣本查找對應格點 × 月份的歷史均值
        anom_vals = []
        lats = df['Latitude'].values
        lons = df['Longitude'].values
        months = df['_month'].values
        obs_vals = df[var].values

        # 建 KD-tree（只需建一次）
        clim_coords = np.radians(clim_df[['Latitude', 'Longitude']].values)
        tree = cKDTree(clim_coords)

        for i in range(len(df)):
            sample_coord = np.radians([[lats[i], lons[i]]])
            _, idx = tree.query(sample_coord, k=1)
            month = months[i]

            clim_row = clim_df.iloc[idx[0]]
            # 若該筆 clim_df 是多月的（月份已展開），需再 filter month
            # 先取對應 month 的 clim 值
            clim_month_val = clim_df[
                (np.abs(clim_df['Latitude'] - lats[i]) < 0.05) &
                (np.abs(clim_df['Longitude'] - lons[i]) < 0.05) &
                (clim_df['month'] == month)
            ][f'{var}_clim']

            if len(clim_month_val) == 0:
                anom_vals.append(np.nan)
            else:
                anom_vals.append(obs_vals[i] - clim_month_val.values[0])

        df[f'{var}_anom'] = anom_vals

    df = df.drop(columns=['_month'])
    anom_cols = [c for c in df.columns if c.endswith('_anom')]
    print(f"[add_anomalies] 完成，新增 {len(anom_cols)} 個距平欄位：{anom_cols}")
    return df


# 向下相容舊版呼叫（合併三步驟）
def apply_feature_engineering(df):
    df = add_basic_derived(df)
    df = add_interactions(df)
    return df


In [ ]:
print("=== 對 Training 資料進行 1+2+4 特徵工程 ===")
Terraclimate_training_df = add_basic_derived(Terraclimate_training_df)
Terraclimate_training_df = add_interactions(Terraclimate_training_df)

vars_for_anom = ['ppt', 'pet', 'aet', 'def', 'pdsi', 'q', 'soil', 'tmax', 'tmin', 'vpd', 'srad']
Terraclimate_training_df = add_anomalies(Terraclimate_training_df, ds, vars_for_anom)

print(f"\n✅ Training 完成！shape: {Terraclimate_training_df.shape}")
Terraclimate_training_df.to_csv("terraclimate_features_training_sonnet_v2.csv", index=False)
display(Terraclimate_training_df.head())


In [8]:
# Terraclimate_training_df['Latitude'] = Water_Quality_df['Latitude']
# Terraclimate_training_df['Longitude'] = Water_Quality_df['Longitude']
# Terraclimate_training_df['Sample Date'] = Water_Quality_df['Sample Date']
# Terraclimate_training_df = Terraclimate_training_df[['Latitude', 'Longitude', 'Sample Date', 'pet']]
# Terraclimate_training_df.to_csv('terraclimate_features_training.csv', index=False)
Terraclimate_training_df.to_csv("terraclimate_features_training_sonnet_v2.csv", index=False)
print("Saved:", Terraclimate_training_df.shape)
Terraclimate_training_df.head()

In [9]:
# Preview File
display(Terraclimate_training_df.head())

In [ ]:
Terraclimate_training_df.to_csv("/tmp/terraclimate_features_training_sonnet_v2.csv",index = False)

In [ ]:
session.sql("""
    PUT file:///tmp/terraclimate_features_training_sonnet_v2.csv
    'snow://workspace/USER$.PUBLIC."EY-AI-and-Data-Challenge"/versions/live/'
    AUTO_COMPRESS=FALSE
    OVERWRITE=TRUE
""").collect()

print("File saved! Refresh the browser to see the files in the sidebar")

**Note:** If you're using your own workspace, remember to replace "EY-AI-and-Data-Challenge" with your workspace name in the file path.

### Extracting features for the validation dataset

In [10]:
Validation_df=pd.read_csv('submission_template.csv')
display(Validation_df.head())

In [11]:
Validation_df.shape

In [12]:
# # Load TerraClimate dataset, filter (time,region,parameter), filter for nearest parameter values
# Terraclimate_validation_df = assign_nearest_climate(Validation_df, tc_parameter, 'pet')
# 確保 ds 與 tc_vars_exist 已在前面 cell11 建好
# ds = load_terraclimate_dataset()
# tc_vars_exist = [...]

Terraclimate_validation_df = Validation_df[["Latitude", "Longitude", "Sample Date"]].copy()

for v in tc_vars_exist:
    tc_parameter = filterg(ds, v)
    Terraclimate_validation_df[v] = assign_nearest_climate(Validation_df, tc_parameter, v)[v]

# 固定欄位順序（可留可不留）
Terraclimate_validation_df = Terraclimate_validation_df[["Latitude", "Longitude", "Sample Date"] + tc_vars_exist]


In [ ]:
print("=== 對 Validation 資料進行相同處理 ===")
Terraclimate_validation_df = add_basic_derived(Terraclimate_validation_df)
Terraclimate_validation_df = add_interactions(Terraclimate_validation_df)
Terraclimate_validation_df = add_anomalies(Terraclimate_validation_df, ds, vars_for_anom)

print(f"\n🎉 完成！ Training shape: {Terraclimate_training_df.shape} | Validation shape: {Terraclimate_validation_df.shape}")
Terraclimate_validation_df.to_csv("terraclimate_features_validation.csv", index=False)
display(Terraclimate_validation_df.head())


In [13]:
# Terraclimate_validation_df['Latitude'] = Validation_df['Latitude']
# Terraclimate_validation_df['Longitude'] = Validation_df['Longitude']
# Terraclimate_validation_df['Sample Date'] = Validation_df['Sample Date']
# Terraclimate_validation_df = Terraclimate_validation_df[['Latitude', 'Longitude', 'Sample Date', 'pet']]
# Terraclimate_validation_df.to_csv('terraclimate_features_validation.csv', index=False)
Terraclimate_validation_df.to_csv("terraclimate_features_validation.csv", index=False)
print("Saved:", Terraclimate_validation_df.shape)


In [14]:
# Preview File
display(Terraclimate_validation_df.head())

In [ ]:
Terraclimate_validation_df.to_csv("/tmp/terraclimate_features_validation_sonnet_v2.csv",index = False)

In [ ]:
session.sql("""
    PUT file:///tmp/terraclimate_features_validation_sonnet_v2.csv
    'snow://workspace/USER$.PUBLIC."EY-AI-and-Data-Challenge"/versions/live/'
    AUTO_COMPRESS=FALSE
    OVERWRITE=TRUE
""").collect()

print("File saved! Refresh the browser to see the files in the sidebar")

**Note:** If you're using your own workspace, remember to replace "EY-AI-and-Data-Challenge" with your workspace name in the file path.

In [ ]:
Terraclimate_validation_df

In [ ]:
Terraclimate_training_df